In [5]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# ============================================================
# GLOBAL NUCLEAR ARSENALS — CURRENT INVENTORY (6/6)
# Plotly version (HTML-ready)
# ============================================================

# --- PATH CONFIG ---
try:
    from const import DATA_DIR, VISUALIZATIONS_DIR
except ImportError:
    DATA_DIR = Path("data")
    VISUALIZATIONS_DIR = Path("visualizations")

VISUALIZATIONS_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================
# 1. LOAD DATA
# ============================================================
file_path = (
    DATA_DIR
    / "nuclear-weapons"
    / "nuclear-warhead-inventories"
    / "nuclear-warhead-inventories.csv"
)

df = pd.read_csv(file_path)
df.columns = [c.strip() for c in df.columns]

country_col = next((c for c in df.columns if c.lower() in ["entity", "country"]), "Entity")
year_col    = next((c for c in df.columns if c.lower() == "year"), "Year")
value_col   = next((c for c in df.columns if "warhead" in c.lower()), None)

# ============================================================
# 2. DATA PROCESSING
# ============================================================
df_latest = (
    df.sort_values(year_col)
      .groupby(country_col)
      .tail(1)
)

df_plot = (
    df_latest[[country_col, value_col]]
    .rename(columns={country_col: "Country", value_col: "Warheads"})
    .dropna()
    .query("Warheads > 0")
    .sort_values("Warheads", ascending=True)
)

# ============================================================
# 3. COLORS — focus USA & Russia
# ============================================================
colors = [
    "#0d5a8a" if c in ["Russia", "United States"] else "#bdbdbd"
    for c in df_plot["Country"]
]

# ============================================================
# 4. BAR CHART
# ============================================================
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=df_plot["Warheads"],
        y=df_plot["Country"],
        orientation="h",
        marker_color=colors,
        text=[f"{int(v):,}" for v in df_plot["Warheads"]],
        textposition="outside",
        textfont=dict(
            size=[12 if c in ["Russia", "United States"] else 11 for c in df_plot["Country"]],
            color="#333"
        ),
        hoverinfo="skip"
    )
)

# ============================================================
# 5. TITLES & CONTEXT
# ============================================================
ref_year = int(df[year_col].max())
total_warheads = int(df_plot["Warheads"].sum())

fig.update_layout(
    title=dict(
        text=f"<b>Global Nuclear Arsenals ({ref_year} Estimates)</b><br>"
             f"<span style='font-size:13px;color:#666;'>"
             f"Total estimated inventory: ~{total_warheads:,} warheads"
             f"</span>",
        x=0.03,
        y=0.95,
        xanchor="left"
    ),
    width=1000,
    height=550,
    margin=dict(l=180, r=40, t=100, b=80),
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="#222"
    ),
    xaxis=dict(
        visible=False
    ),
    yaxis=dict(
        ticks="",
        showline=False
    )
)

# ============================================================
# 6. LIGHT INSIGHT (DESCRITTIVO)
# ============================================================
top2 = df_plot[df_plot["Country"].isin(["Russia", "United States"])]["Warheads"].sum()
share = top2 / total_warheads * 100
max_val = df_plot["Warheads"].max()

fig.add_annotation(
    x=max_val * 0.55,
    y=1.4,
    text=f"Russia and the United States<br>"
         f"account for ≈{share:.0f}% of the global arsenal",
    showarrow=False,
    font=dict(size=13, color="#0d5a8a"),
    align="left"
)

# ============================================================
# 7. DATA NOTE
# ============================================================
fig.add_annotation(
    x=0,
    y=-0.12,
    xref="paper",
    yref="paper",
    text="Only countries with publicly available inventory estimates are shown.",
    showarrow=False,
    font=dict(size=10, color="#888"),
    align="left"
)



In [6]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "global_nuclear_arsenals_today.html")